In [1]:
import numpy as np
import pandas as pd

In [2]:
# Пытаюсь а) нарисовать график по таймдельтам б) привести таймдельты к матплотлибоугодному виду

(task_01
    .fillna (pd.to_timedelta(0))
    .applymap (lambda x: pd.to_timedelta(x.round('s')))
    # .applymap (lambda x: x.as_unit('s'))    # 'Timedelta' object has no attribute 'as_unit' видимо более новое
    # .apply (lambda col: col.to_pytimedelta(), axis=0)
    # .applymap (lambda x: x.to_numpy())          # timedelta64[ns]
    # .applymap (lambda x: x.to_pytimedelta())    #  timedelta64[ns]
    # .applymap (lambda x: pd.to_timedelta(x))    # timedelta64[ns]
    .dtypes
    # .plot(kind = 'barh')
)
# task_01[task_01.columns[0]].value_counts()

NameError: name 'task_01' is not defined

In [5]:
np.arange(3)

array([0, 1, 2])

In [ ]:
# Задание 1. Подготовка таблицы №1

# Наброски ответов на вопросы
# Вопрос 1. Отличается ли время прохождения различных этапов у пользователей, которые прошли обучение, и пользователей, не начинавших обучение?
task_01 = \
(user_events_wide
    .pivot_table
    (   index = ['tutorial_status']
        , values = [col for col in user_events_wide.columns if 'gap' in col]
        # , columns = ['event_type']
        , aggfunc = 'mean'
        , sort = False
    )
)
task_01

In [ ]:
# Задание 1. Подготовка таблицы №2

task_01_alt = \
(   user_events_wide
    .set_index ('tutorial_status')
    .filter (like='gap')
    .melt(ignore_index=False, var_name='event', value_name='interval')
    .assign
    (**{'event': lambda df: 
            df['event']
            .str.split(':')
            .map(lambda x:x[1])
            .astype(pd.api.types.CategoricalDtype(categories=events_arranged, ordered=True))
    })
    .groupby (['tutorial_status', 'event']).mean()
    .reset_index()
    .assign
    (**{'interval_hours': lambda df: 
                df['interval'].map(lambda x: x.days*24 + x.seconds/3600).round(3)
        , 'percent_to_max': lambda df:
                (df['interval_hours'] / df.groupby('event')['interval_hours'].transform(max)).round(3)*100
    })
    # .dropna()
)
task_01_alt

In [ ]:
# Задание 1. Подход к графику №1

def humanize_headers (df):
    df.columns = \
    (   df.columns
        .str.replace('_',' ')
        .str.split(':')
        .map(lambda x:x[1])
    )
    return df
plot_01 = \
(task_01
    # .fillna (pd.to_timedelta(0))
    .applymap (lambda x: x.days*24 + x.seconds/3600).round(2)
    .apply (lambda x: x/x.sum())
    .pipe (humanize_headers)
    .transpose()
    .rename_axis(columns=None, index='event')
    .reset_index()
    # .plot(kind = 'barh')
)
plot_01

In [ ]:
# Задание 1. Подход к графику №2
c = \
(   task_01_alt[['event', 'tutorial_status', 'percent_to_max']]
    .set_index (['event', 'tutorial_status'])
    .unstack ('tutorial_status')
    .sort_index(ascending = False, axis = 1)
    .plot
    (   kind = 'barh'
        , y = 'percent_to_max'
        , stacked = False
    )
)
c

In [ ]:
# Задание 1. Подход к графику №3 + изучение Сиборна

b = sns.barplot\
(   data = task_01_alt
    , y = 'event'
    , x = 'percent_to_max'
    , hue = 'tutorial_status'
)


b.__class__     # matplotlib.axes._axes.Axes
# sns.barplot возвращает объект matplotlib.axes
# у этого объекта есть свойство containers
b.containers.__class__  # list
b.containers.__len__ () # 3
# containers моего графика представляет собой список из 3 элементов (соответствуют числу ПАРАМЕТРОВ в моей классификации)
b.containers[0]         # <BarContainer object of 6 artists>
b.containers[0].__class__   # matplotlib.container.BarContainer
# каждый container включает в себя 6 (по числу записей) "рисовальщиков"
b.containers[2].datavalues
# array([  nan,  83.4, 100. ,  44.3,  97.8,  88.6])
b.containers[2].get_label() # 'finished'

for c in b.containers:
    b.bar_label (c, label_type = 'edge', padding = -25, color = 'white')






# b.bar_label ()
# b.__class__ # matplotlib.axes._axes.Axes
